# Домашнее задание к лекции "Базовые понятия статистики"

## Обязательная часть

Будем осуществлять работу с непростым [набором данных](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv) о состоянии здоровья лошадей, испытывающих кишечные колики. 

### Задание 1. Базовое изучение

Изучить представленный набор данных на основе [описания его столбцов](https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.names) и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.

### Задание 2. Работа с выбросами

В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

### Задание 3. Работа с пропусками

Рассчитать количество выбросов для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [1]:
import pandas as pd
import numpy as np
horses = pd.read_csv('../stat_basics/horse_data.csv', na_values='?')

In [2]:
horses.head()

,2,1,530101,38.50,66,28,3,3.1,?,2.1,...,45.00,8.40,?.4,?.5,2.2,2.3,11300,00000,00000.1,2.4
0,1.0,1,534817,39.2,88.0,20.0,NaN,NaN,4.0,1.0,...,50.0,85.0,2.0,2.0,3.0,2,2208,0,0,2
1,2.0,1,530334,38.3,40.0,24.0,1.0,1.0,3.0,1.0,...,33.0,6.7,NaN,NaN,1.0,2,0,0,0,1
2,1.0,9,5290409,39.1,164.0,84.0,4.0,1.0,6.0,2.0,...,48.0,7.2,3.0,5.3,2.0,1,2208,0,0,1
3,2.0,1,530255,37.3,104.0,35.0,NaN,NaN,6.0,2.0,...,74.0,7.4,NaN,NaN,2.0,2,4300,0,0,2
4,2.0,1,528355,NaN,NaN,NaN,2.0,1.0,3.0,1.0,...,NaN,NaN,NaN,NaN,1.0,2,0,0,0,2


In [3]:
horses.shape

(299, 28)

In [4]:
df = horses.iloc[:,[0,1,2,3,4,5,19,22,23,24,25,26]]
df.columns = ['Surgery', 'Age', 'Hospital_id', 'Temperature', 'Pulse', 'Respiratore_rate', 'total_prot', 'outcome', 'surgical_lesion', 'lession_type', 'lession_type_2', 'lession_type_3']
# 0 - операция
# 1 - возраст 1 - взрослая
# 2 - id при лечении
# 3 - температура
# 4 - пульс
# 5 - частота дыхания
# 19 - общий белок норма (6-7.5)
# 22 - исход 1 - жива 2-не жива 3-усыпили
# 23 - хирургическое повреждение?
# 24, 25, 26 - место поражения
#            - тяжесть поражения
#            - подтип
#            - конкретный код
df

,Surgery,Age,Hospital_id,Temperature,Pulse,Respiratore_rate,total_prot,outcome,surgical_lesion,lession_type,lession_type_2,lession_type_3
0,1.0,1,534817,39.2,88.0,20.0,85.0,3.0,2,2208,0,0
1,2.0,1,530334,38.3,40.0,24.0,6.7,1.0,2,0,0,0
2,1.0,9,5290409,39.1,164.0,84.0,7.2,2.0,1,2208,0,0
3,2.0,1,530255,37.3,104.0,35.0,7.4,2.0,2,4300,0,0
4,2.0,1,528355,NaN,NaN,NaN,NaN,1.0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
294,1.0,1,533886,NaN,120.0,70.0,65.0,3.0,2,3205,0,0
295,2.0,1,527702,37.2,72.0,24.0,NaN,3.0,1,2208,0,0
296,1.0,1,529386,37.5,72.0,30.0,6.8,2.0,1,3205,0,0
297,1.0,1,530612,36.5,100.0,24.0,6.0,1.0,1,2208,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 12 columns):
Surgery             298 non-null float64
Age                 299 non-null int64
Hospital_id         299 non-null int64
Temperature         239 non-null float64
Pulse               275 non-null float64
Respiratore_rate    241 non-null float64
total_prot          266 non-null float64
outcome             298 non-null float64
surgical_lesion     299 non-null int64
lession_type        299 non-null int64
lession_type_2      299 non-null int64
lession_type_3      299 non-null int64
dtypes: float64(6), int64(6)
memory usage: 28.2 KB


### % пропусков для каждого столбца

In [6]:
(df.isna().mean()*100).round(2)

Surgery              0.33
Age                  0.00
Hospital_id          0.00
Temperature         20.07
Pulse                8.03
Respiratore_rate    19.40
total_prot          11.04
outcome              0.33
surgical_lesion      0.00
lession_type         0.00
lession_type_2       0.00
lession_type_3       0.00
dtype: float64

### Surgery

In [7]:
df['Surgery'].value_counts()

1.0    180
2.0    118
Name: Surgery, dtype: int64

В результатах наблюдений больше прооперированных лошадей. Так же имеется одно пропущенное значние. 

In [8]:
df['Age'].value_counts()

1    275
9     24
Name: Age, dtype: int64

Тут возникает вопрос к сбору данных. Так как в документации написано, что в данном столбце могут быть значения только 1 или 2. За что принимать 9 вопрос филосовский. 

### Hospital id

In [9]:
df['Hospital_id'].value_counts()

529796    2
528931    2
528890    2
527544    2
528151    2
         ..
527698    1
530255    1
530254    1
533836    1
535043    1
Name: Hospital_id, Length: 283, dtype: int64

In [10]:
df['Hospital_id'].value_counts().value_counts()

1    267
2     16
Name: Hospital_id, dtype: int64

Имеются лошади, которые были прооперированы дважды

### Temperature

In [11]:
print('max:', df['Temperature'].max())
print('min:', df['Temperature'].min())
print('mode:', df['Temperature'].mode()[0])
print('median:', df['Temperature'].median())
print('пропусков:',(df['Temperature'].isna().mean()*100).round(2), '%')

max: 40.8
min: 35.4
mode: 38.0
median: 38.2
пропусков: 20.07 %


Заполнение пропусков модой для прооперированных и непрооперированных лошадей, в зависимости от типа операции. 

In [12]:
print(df.groupby(['Surgery', (df.lession_type)//1000])['Temperature'].median())

Surgery  lession_type
1.0      0               38.20
         1               39.40
         2               38.25
         3               38.00
         4               37.75
         5               38.00
         6               38.10
         7               38.40
         8               38.00
         9               38.20
         11              40.80
         12              37.10
         21              39.30
         31              38.50
2.0      0               38.20
         1               38.05
         2               37.80
         3               38.40
         4               38.50
         5               38.60
         7               40.30
         8               39.10
         9               39.00
         31              37.20
         41                NaN
Name: Temperature, dtype: float64


In [13]:
df['Temperature'].fillna(df.groupby(['Surgery', (df.lession_type)//1000])['Temperature'].transform('median'), inplace=True)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [14]:
print('max:', df['Temperature'].max())
print('min:', df['Temperature'].min())
print('mode:', df['Temperature'].mode()[0])
print('median:', df['Temperature'].median())
print('пропусков:',(df['Temperature'].isna().mean()*100).round(2), '%')

max: 40.8
min: 35.4
mode: 38.0
median: 38.2
пропусков: 0.33 %


Судя по медианному значению, температура большенства лошадей повышена (при норме в 37.8)

### Pulse

In [15]:
df.Pulse.describe()

count    275.000000
mean      71.934545
std       28.680522
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: Pulse, dtype: float64

In [16]:
print(df.groupby(['Surgery', 'outcome'])['Pulse'].median())

Surgery  outcome
1.0      1.0        60.0
         2.0        84.0
         3.0        80.0
2.0      1.0        52.0
         2.0        92.0
         3.0        85.0
Name: Pulse, dtype: float64


In [17]:
print(df.groupby([(df.lession_type)//1000])['Pulse'].median().sort_values(ascending = False))

lession_type
9     137.0
1      98.0
41     96.0
7      92.0
4      88.0
12     75.0
11     74.0
2      72.0
31     65.0
21     64.0
5      62.0
6      60.0
3      60.0
8      56.0
0      50.0
Name: Pulse, dtype: float64


Исходя из данных выше, можно сделать вывод, что у перенесших операцию и выживших пульс выше, но те лошади, которые проходи лечение без операции и в дальнейшем не выжили имеют температуру выше, чем те, которые перенесли операцию. После операций на желудок(гастрит), кишечник и мочевой пузырь пульс многократно превышает норму. Заполним пропуски для каждой группы операций

In [18]:
df['Pulse'].fillna(df.groupby([(df.lession_type)//1000])['Pulse'].transform('median'), inplace=True)

### Respiratore rate

In [19]:
df.Respiratore_rate.describe()

count    241.000000
mean      30.427386
std       17.678256
min        8.000000
25%       18.000000
50%       24.000000
75%       36.000000
max       96.000000
Name: Respiratore_rate, dtype: float64

In [20]:
print(df.groupby(['Pulse'])['Respiratore_rate'].median().sort_values(ascending = False).head(10))

Pulse
146.0    96.0
184.0    84.0
164.0    84.0
124.0    62.0
150.0    61.0
108.0    51.0
136.0    48.0
129.0    48.0
120.0    48.0
140.0    46.0
Name: Respiratore_rate, dtype: float64


In [21]:
df = df.sort_values('Pulse')
df.fillna(method = 'ffill', axis = 'index', inplace = True)

Много пропусков, так же очень большие колебания, так что полезность данных вызывает сомнения. Очевидно, что частота дыхания зависит от пульса, так что можно заполнить пропуски соседними значениями. 

### total protein

In [22]:
df.total_prot.describe()

count    299.000000
mean      23.953846
std       27.113140
min        3.300000
25%        6.550000
50%        7.500000
75%       56.000000
max       89.000000
Name: total_prot, dtype: float64

In [23]:
df.groupby([(df.lession_type)//1000])['total_prot'].median()

lession_type
0      8.60
1      8.15
2      7.60
3      7.30
4      7.20
5      6.55
6     65.00
7      6.50
8      6.60
9      8.00
11     7.40
12     7.40
21     6.70
31     6.70
41     8.50
Name: total_prot, dtype: float64

In [24]:
df['total_prot'].fillna(df.groupby([(df.lession_type)//1000])['total_prot'].transform('median'), inplace=True)

После операций по пересадки слепой и толстой кишки значение общего белка многократно превышает норму (6.5-7). Чем выше это значение, тем сильнее обезвоживание, что может привести к гибели животного. 

### Outcome

In [25]:
df.groupby([(df.lession_type)//1000])['outcome'].describe()

,count,mean,std,min,25%,50%,75%,max
lession_type,,,,,,,,
0,62.0,1.048387,0.282113,1.0,1.00,1.0,1.00,3.0
1,10.0,2.300000,0.674949,1.0,2.00,2.0,3.00,3.0
2,78.0,1.679487,0.781168,1.0,1.00,1.0,2.00,3.0
3,79.0,1.607595,0.774890,1.0,1.00,1.0,2.00,3.0
4,22.0,1.681818,0.646335,1.0,1.00,2.0,2.00,3.0
5,12.0,1.666667,0.778499,1.0,1.00,1.5,2.00,3.0
6,5.0,1.800000,1.095445,1.0,1.00,1.0,3.00,3.0
7,12.0,2.000000,0.738549,1.0,1.75,2.0,2.25,3.0
8,3.0,1.333333,0.577350,1.0,1.00,1.0,1.50,2.0


Больше половинв операций на кишечник и гастрит приводят к гибели/усыплению животного.

### Проверка на пустые значения

In [26]:
(df.isna().mean()*100).round(2)

Surgery             0.0
Age                 0.0
Hospital_id         0.0
Temperature         0.0
Pulse               0.0
Respiratore_rate    0.0
total_prot          0.0
outcome             0.0
surgical_lesion     0.0
lession_type        0.0
lession_type_2      0.0
lession_type_3      0.0
dtype: float64

### Работа с выбросами

In [27]:
df

,Surgery,Age,Hospital_id,Temperature,Pulse,Respiratore_rate,total_prot,outcome,surgical_lesion,lession_type,lession_type_2,lession_type_3
231,1.0,1,5262543,38.50,30.0,18.0,7.7,1.0,1,2113,0,0
129,1.0,1,528964,37.80,30.0,12.0,7.7,2.0,1,2113,0,0
202,2.0,1,529685,37.20,36.0,9.0,5.7,1.0,2,31110,0,0
275,1.0,1,534787,38.00,36.0,16.0,75.0,3.0,2,3111,0,0
241,1.0,1,530354,38.25,36.0,32.0,5.9,2.0,1,2205,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
40,2.0,9,5288249,39.00,150.0,72.0,8.5,1.0,1,9400,0,0
274,1.0,9,5297159,38.80,150.0,50.0,6.2,2.0,1,4207,0,0
54,1.0,9,5282839,38.60,160.0,20.0,6.2,2.0,1,7111,0,0
2,1.0,9,5290409,39.10,164.0,84.0,7.2,2.0,1,2208,0,0


### Temperature

In [28]:
q1 = df['Temperature'].quantile(0.25)
q3 = df['Temperature'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['Temperature'].between(lower_bound, upper_bound, inclusive=True)]
len(remove_outliers)

281

In [29]:
pd.concat([df, remove_outliers]).drop_duplicates(keep=False)

,Surgery,Age,Hospital_id,Temperature,Pulse,Respiratore_rate,total_prot,outcome,surgical_lesion,lession_type,lession_type_2,lession_type_3
250,2.0,1,527940,36.6,42.0,18.0,7.1,2.0,1,5111,0,0
140,2.0,1,522979,36.0,42.0,30.0,6.8,2.0,2,1400,0,0
258,1.0,1,527734,40.8,72.0,42.0,7.4,2.0,1,11400,0,0
19,1.0,1,530157,39.9,72.0,60.0,6.1,1.0,1,2111,0,0
73,1.0,9,5292929,40.8,74.0,30.0,4.9,2.0,1,11124,0,0
273,1.0,1,534624,40.8,76.0,18.0,11.0,3.0,1,11124,0,0
117,1.0,1,533983,36.5,78.0,30.0,75.0,1.0,1,3112,6112,0
280,2.0,1,530624,40.0,78.0,30.0,6.5,2.0,1,8400,0,0
237,2.0,1,528702,36.1,88.0,16.0,7.0,3.0,1,2209,0,0
79,1.0,1,527518,36.4,98.0,35.0,6.4,2.0,1,2205,0,0


Для температуры нет ни каких аномальных значений, значит данные необходимо оставить

### Pulse

In [30]:
q1 = df['Pulse'].quantile(0.25)
q3 = df['Pulse'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['Pulse'].between(lower_bound, upper_bound, inclusive=True)]
len(remove_outliers)

294

In [31]:
pd.concat([df, remove_outliers]).drop_duplicates(keep=False)

,Surgery,Age,Hospital_id,Temperature,Pulse,Respiratore_rate,total_prot,outcome,surgical_lesion,lession_type,lession_type_2,lession_type_3
40,2.0,9,5288249,39.0,150.0,72.0,8.5,1.0,1,9400,0,0
274,1.0,9,5297159,38.8,150.0,50.0,6.2,2.0,1,4207,0,0
54,1.0,9,5282839,38.6,160.0,20.0,6.2,2.0,1,7111,0,0
2,1.0,9,5290409,39.1,164.0,84.0,7.2,2.0,1,2208,0,0
254,1.0,9,5294539,38.8,184.0,84.0,3.3,2.0,1,7111,0,0


Для пульса так же значения вполне реальны. 

### Respiratore rate

In [32]:
q1 = df['Respiratore_rate'].quantile(0.25)
q3 = df['Respiratore_rate'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['Respiratore_rate'].between(lower_bound, upper_bound, inclusive=True)]
len(remove_outliers)

276

In [33]:
pd.concat([df, remove_outliers]).drop_duplicates(keep=False)

,Surgery,Age,Hospital_id,Temperature,Pulse,Respiratore_rate,total_prot,outcome,surgical_lesion,lession_type,lession_type_2,lession_type_3
124,1.0,1,528461,38.00,42.0,68.0,7.6,1.0,1,2205,0,0
105,1.0,1,528668,38.30,52.0,96.0,6.1,1.0,1,3209,0,0
119,1.0,1,528682,39.40,54.0,66.0,6.0,1.0,1,2206,0,0
264,2.0,1,528590,38.60,56.0,68.0,6.0,3.0,1,5206,0,0
83,1.0,1,529849,37.80,60.0,80.0,4.5,1.0,1,5206,0,0
185,1.0,1,5278331,39.30,64.0,90.0,6.7,1.0,1,21110,0,0
268,1.0,1,527883,39.30,64.0,90.0,6.7,1.0,1,31110,0,0
207,1.0,1,535338,37.80,88.0,80.0,89.0,3.0,1,3205,0,0
159,1.0,1,527916,37.75,88.0,80.0,58.0,1.0,1,4205,0,0
39,1.0,1,529498,37.75,88.0,80.0,6.5,2.0,1,4205,0,0


Очень много значений превышающих норму (8-10) в разы. Вопрос к точности сбора данных. 

### total prot

In [34]:
q1 = df['total_prot'].quantile(0.25)
q3 = df['total_prot'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df['total_prot'].between(lower_bound, upper_bound, inclusive=True)]
print(iqr)

49.45


In [35]:
pd.concat([df, remove_outliers]).drop_duplicates(keep=False)

,Surgery,Age,Hospital_id,Temperature,Pulse,Respiratore_rate,total_prot,outcome,surgical_lesion,lession_type,lession_type_2,lession_type_3


Получился достаточно большой межквантильный размах в следствии чего, все данные попали в норму. По данному методу поиска, выбросов не обнаружено. 

## Дополнительная часть (необязательная)

Выполнить задания 1-3 для всего набора данных.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам
Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.